# Capítulo 12 - Redes Neurais Recorrentes

## 12.2. Modelos de Linguagem Baseados em Redes Neurais Recorrentes

O objetivo deste notebook consiste em desenvolver modelos de linguagem baseados em redes neurais recorrentes. Iremos abordar dois tipos de modelos:

*   word-to-word: trata cada palavra do *corpus* como um documento. O processo de treinamento consiste em pares (palavra,próxima palavra) como sendo o texto e o rótulo;
*   sentence-to-word: considera as palavras anteriores de uma sentença (e suas relações de dependência) do *corpus* para prever a próxima palavra.



In [82]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,Activation,Flatten,Dropout,Bidirectional
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [83]:
import importlib
import cap12_2_lang_models_rnn as myUtils
importlib.reload(myUtils)

<module 'cap12_2_lang_models_rnn' from 'C:\\Users\\Guilherme\\Documents\\Programming\\Python\\Python_Projects\\DataScience\\NLP\\cap12_2_lang_models_rnn.py'>

In [84]:
corpus = "Quero jogar futebol hoje\n Hoje não tem futebol\n"
corpus = corpus.lower()
corpus

'quero jogar futebol hoje\n hoje não tem futebol\n'

### Modelo de Linguagem do tipo Word-to-Word

In [85]:
vocabulary = {}
tokens = []
for sentence in corpus.split('\n'): # "quero jogar futebol hoje"
    for word in sentence.split(): # # "quero"
        tokens.append(word)
        if word in vocabulary:
            vocabulary[word]+=1
        else:
            vocabulary[word]=1
vocab_size = len(vocabulary)

print("vocabulary", vocabulary, "length:",len(vocabulary))
print("tokens", tokens, "length:",len(tokens))

vocabulary {'quero': 1, 'jogar': 1, 'futebol': 2, 'hoje': 2, 'não': 1, 'tem': 1} length: 6
tokens ['quero', 'jogar', 'futebol', 'hoje', 'hoje', 'não', 'tem', 'futebol'] length: 8


In [86]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(vocabulary)
word2index = tokenizer.word_index
word2index['<OOV>'] = 0
word2index

{'quero': 1,
 'jogar': 2,
 'futebol': 3,
 'hoje': 4,
 'não': 5,
 'tem': 6,
 '<OOV>': 0}

In [87]:
index2word = {}
for key in word2index: # "quero"
    value = word2index[key] # "1"
    index2word[value] = key # "1":"quero"

index2word

{1: 'quero',
 2: 'jogar',
 3: 'futebol',
 4: 'hoje',
 5: 'não',
 6: 'tem',
 0: '<OOV>'}

In [88]:
# tokens[0] = "quero"
X_train = [0]
y_train = [word2index[tokens[0]]] # [1]
for i in range(0,len(tokens)-1): # 0 a 7
    X_train.append(word2index[tokens[i]])
    y_train.append(word2index[tokens[i+1]])
X_train.append(word2index[tokens[len(tokens)-1]])
y_train.append(0)

In [89]:
print(f"{vocabulary=}, {len(vocabulary)=}")
print("tokens", tokens, "length:",len(tokens))
print("word2index:", word2index, "length:",len(word2index))
print(f"{corpus=}")

vocabulary={'quero': 1, 'jogar': 1, 'futebol': 2, 'hoje': 2, 'não': 1, 'tem': 1}, len(vocabulary)=6
tokens ['quero', 'jogar', 'futebol', 'hoje', 'hoje', 'não', 'tem', 'futebol'] length: 8
word2index: {'quero': 1, 'jogar': 2, 'futebol': 3, 'hoje': 4, 'não': 5, 'tem': 6, '<OOV>': 0} length: 7
corpus='quero jogar futebol hoje\n hoje não tem futebol\n'


In [90]:
X_train

[0, 1, 2, 3, 4, 4, 5, 6, 3]

In [91]:
y_train

[1, 2, 3, 4, 4, 5, 6, 3, 0]

In [92]:
model = Sequential()
# input_length=1 => word
# input_dim=vocab_size => vocabulary size

# TODO: why input_dim = vocab_size + 1? i think should be size 1
model.add(Embedding(input_dim=vocab_size+1,output_dim=32,input_length=1))
model.add(Bidirectional(LSTM(256,activation='relu')))
model.add(Dropout(0.5))
model.add(Dense(vocab_size+1,activation='softmax'))

In [93]:
sgd = SGD(learning_rate = 0.001)
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=16,epochs=3)

Epoch 1/3
1/1 [==============================] - 3s 3s/step - loss: 1.9456 - accuracy: 0.1111
Epoch 2/3
1/1 [==============================] - 0s 34ms/step - loss: 1.9474 - accuracy: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 27ms/step - loss: 1.9458 - accuracy: 0.2222


In [94]:
import numpy as np

frase = 'futebol quero jogar'

for w in frase.split(): # "futebol"
    idx = word2index[w] # 3
    prob = model.predict([idx]) # [[0.14256176 0.14265485 0.14337164 0.14302562 0.14251715 0.14291811,  0.14295083]]
    pal = np.argmax(prob) # [2] (indice da maior probabilidade)
    print(f'Palavra atual: {index2word[idx]} Proxima palavra: {index2word[pal]}')


1/1 [==============================] - 0s 357ms/step
Palavra atual: futebol Proxima palavra: não
1/1 [==============================] - 0s 45ms/step
Palavra atual: quero Proxima palavra: <OOV>
1/1 [==============================] - 0s 44ms/step
Palavra atual: jogar Proxima palavra: tem


### Modelo de Linguagem do tipo Sentence-2-Word

Nesse tipo de modelo, as palavras são analisadas dentro das sentenças:

In [95]:
novel_corpus = []
y_train = []
for sentence in corpus.split('\n'): # "quero jogar futebol hoje"

    novos_termos = sentence.split()

    for i in range(0,len(novos_termos)): # 0 a 3
        lista = novos_termos[:i+1]
        novel_corpus.append(lista)
        if i < len(novos_termos)-1: # i < 3
            y_train.append(word2index[novos_termos[i+1]])
        else:
            y_train.append(0)

In [96]:
novel_corpus

[['quero'],
 ['quero', 'jogar'],
 ['quero', 'jogar', 'futebol'],
 ['quero', 'jogar', 'futebol', 'hoje'],
 ['hoje'],
 ['hoje', 'não'],
 ['hoje', 'não', 'tem'],
 ['hoje', 'não', 'tem', 'futebol']]

In [97]:
y_train

[2, 3, 4, 0, 5, 6, 3, 0]

In [98]:
max_length = max([len(sentence) for sentence in novel_corpus])

In [ ]:
max_length

In [100]:
train_sequences = tokenizer.texts_to_sequences(novel_corpus)

In [101]:
train_sequences

[[1], [1, 2], [1, 2, 3], [1, 2, 3, 4], [4], [4, 5], [4, 5], [4, 5, 3]]

In [102]:
trunc_type = 'post'
padding_type = 'pre'

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [103]:

train_padded

array([[0, 0, 0, 1],
       [0, 0, 1, 2],
       [0, 1, 2, 3],
       [1, 2, 3, 4],
       [0, 0, 0, 4],
       [0, 0, 4, 5],
       [0, 0, 4, 5],
       [0, 4, 5, 3]])

In [116]:
model2 = Sequential()
model2.add(Embedding(input_dim=vocab_size+1,output_dim=32,input_length=max_length))
model2.add(Bidirectional(LSTM(256,activation='relu')))
model2.add(Dropout(0.5))
model2.add(Dense(vocab_size+1,activation='softmax'))

In [117]:
sgd = SGD(learning_rate = 0.001)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_padded,y_train,batch_size=16,epochs=3)

Epoch 1/3
1/1 [==============================] - 3s 3s/step - loss: 1.9466 - accuracy: 0.1250
Epoch 2/3
1/1 [==============================] - 0s 62ms/step - loss: 1.9465 - accuracy: 0.2500
Epoch 3/3
1/1 [==============================] - 0s 66ms/step - loss: 1.9448 - accuracy: 0.3750


## Modelo de Linguagem com Corpus da Reuters

**Homework:** desenvolver um modelo de linguagem do tipo Sentence-to-Word utilizando o vocabulário do *corpus* (split do treinamento) da reuters. Aproveite os splits de validação e de testes para experimentos.

Obs.: delimitar o ```vocab_size``` para que seja possível a execução desse notebook no Google Colab. 

In [51]:
from keras.datasets import reuters

vocab_size = 3000

(x_train,y_train_int),(x_test2,y_test2) = reuters.load_data(num_words=vocab_size,test_split=0.3)
word2index = reuters.get_word_index()

In [115]:
index2word = {}

for key,value in word2index.items():
  index2word[value] = key

print(' '.join([index2word[x] for x in x_train[0]]))

KeyError: 8

Lembre-se de construir as variáveis apropriadamente as variáveis ``` y_train ``` e ``` X_train ``` para treinamento dos modelos.

